In [1]:
import os
import ast
import pandas as pd

# ===== Settings =====
input_file = "1_MatDX_EF.xlsx"                   # Input Excel file path
input_sheet = "1_MatDX_EF"                       # Sheet name containing the original data
output_file = "1_MatDX_EF_modified.xlsx"   # Output file (new file)
output_sheet = "Sheet1"                  # Sheet name to save

# ===== 1) Read original sheet =====
df = pd.read_excel(input_file, sheet_name=input_sheet)

# ===== 2) Convert formation_energy string -> dict (safe parsing) =====
def to_dict_safe(x):
    if isinstance(x, dict):
        return x
    if pd.isna(x):
        return {}
    try:
        return ast.literal_eval(str(x))
    except Exception:
        return {}

if "formation_energy" not in df.columns or "formula" not in df.columns:
    raise KeyError("Required columns (formula, formation_energy) are missing.")

fe_series = df["formation_energy"].apply(to_dict_safe)

# ===== 3) Create new dataframe =====
def get_val(d, key):
    try:
        return d.get(key, None)
    except Exception:
        return None

new_data = pd.DataFrame({
    "file_name": [f"{i+2}_{str(f)}.cif" for i, f in enumerate(df["formula"])],
    "value_per_atom": [get_val(d, "value_per_atom") for d in fe_series],
    "value": [get_val(d, "value") for d in fe_series],
})

# ===== 4) Save to new file (only with processed sheet) =====
with pd.ExcelWriter(output_file, engine="openpyxl", mode="w") as writer:
    new_data.to_excel(writer, sheet_name=output_sheet, index=False)

print(f"Done: '{output_file}' has been created with only the '{output_sheet}' sheet.")


Done: '1_MatDX_EF_modified.xlsx' has been created with only the 'Sheet1' sheet.
